### Pytorch
Pytorch는 TensorFlow와 함께 Deep Learning에서 가장 널리 사용되는 framework입니다.  

초기에는 Torch라는 이름으로 Lua 언어 기반으로 만들어졌으나, 이후 python 기반으로 변경한 것이 Pytorch입니다.  

New York 대학교와 Facebook이 공동으로 만들었고, Deep Learning 연구자들 사이에서는 가장 대중적으로 널리 사용되는 framework입니다.  

In [1]:
import torch

print(torch.__version__)

1.9.0.post2


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [4]:
# MNIST Data

# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.MNIST(
        root = 'data',
        train = True ,
        download = True,
        transform = ToTensor(),

)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [5]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data , batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X , y in test_dataloader :
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [6]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cpu device


In [12]:
# 모델 정의

class  NeuralNetwork(nn.Module):
    def __init__(self) :
        super(NeuralNetwork , self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28 , 128),
                nn.ReLU(),
                nn.Dropout(0,2),
                nn.Linear(128, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0, inplace=2)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [13]:
# Loss 함수와 Optimizer 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [14]:
# Training을 위한 함수
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [15]:
# Test를 위한 함수
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.318804  [    0/60000]
loss: 0.412134  [ 6400/60000]
loss: 0.275046  [12800/60000]
loss: 0.335593  [19200/60000]
loss: 0.214580  [25600/60000]
loss: 0.328789  [32000/60000]
loss: 0.142627  [38400/60000]
loss: 0.344883  [44800/60000]
loss: 0.262615  [51200/60000]
loss: 0.303591  [57600/60000]
Test Error: 
 Accuracy: 94.1%, Avg loss: 0.199658 

Epoch 2
-------------------------------
loss: 0.146446  [    0/60000]
loss: 0.243895  [ 6400/60000]
loss: 0.110436  [12800/60000]
loss: 0.194621  [19200/60000]
loss: 0.156749  [25600/60000]
loss: 0.256668  [32000/60000]
loss: 0.068056  [38400/60000]
loss: 0.245123  [44800/60000]
loss: 0.170067  [51200/60000]
loss: 0.217406  [57600/60000]
Test Error: 
 Accuracy: 95.7%, Avg loss: 0.142595 

Epoch 3
-------------------------------
loss: 0.082124  [    0/60000]
loss: 0.180853  [ 6400/60000]
loss: 0.086474  [12800/60000]
loss: 0.118662  [19200/60000]
loss: 0.116895  [25600/60000]
loss: 0.191546  [32000/600